### Find Station Venues

Extract venue information for all stations in Ottawa and Toronto.

### Install Libraries

In [1]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

import folium # map rendering library

print('Libraries imported.')

Libraries imported.


### Import Data

In [2]:
Stations_Location = pd.read_csv('Stations.csv')
print(Stations_Location.head())

CategoryKey = pd.read_csv('CategoryKey.csv')
print(CategoryKey.head())

     City     Station   Latitude  Longitude
0  Ottawa       Blair  45.431026 -75.608416
1  Ottawa    Cyrville  45.422745 -75.626372
2  Ottawa  St-Laurent  45.420605 -75.638188
3  Ottawa    Tremblay  45.416933 -75.653348
4  Ottawa     Hurdman  45.412357 -75.664320
                 Parent            categories
0  Arts & Entertainment  Arts & Entertainment
1  Arts & Entertainment          Amphitheater
2  Arts & Entertainment              Aquarium
3  Arts & Entertainment                Arcade
4  Arts & Entertainment           Art Gallery


### Venue Categories


Define Foursquare Credentials

In [3]:
CLIENT_ID = 'IP5SLL30YW00QF4AF4WCUSTRX4FHKJZGGYMRHFXNFN1TCYPM' # your Foursquare ID
CLIENT_SECRET = 'BTECFQLVC11ZLJPRVKA1S3YNYHXOZRS02U5W4ZB4WJP33HWB' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: IP5SLL30YW00QF4AF4WCUSTRX4FHKJZGGYMRHFXNFN1TCYPM
CLIENT_SECRET:BTECFQLVC11ZLJPRVKA1S3YNYHXOZRS02U5W4ZB4WJP33HWB


Define Function To Extract Categories From Venues

In [4]:
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

Define and Make Foursquare API Request For Station Venues and Group Venues by Sub-Category

In [5]:
Station_dfs = {}

radius = 416 # in meters; 5 min walk
LIMIT = 1000


for i in range(Stations_Location.shape[0]):
        
    latitude = Stations_Location.iloc[i,2]
    longitude = Stations_Location.iloc[i,3]
    
    url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, radius, LIMIT)
    
    results = requests.get(url).json()
    
    venues = results['response']['venues']
    
    nearby_venues = json_normalize(venues) # flatten JSON

    # filter columns
    filtered_columns = ['name', 'categories', 'location.lat', 'location.lng']
    nearby_venues =nearby_venues.loc[:, filtered_columns]

    # filter the category for each row
    nearby_venues['categories'] = nearby_venues.apply(get_category_type, axis=1)

    # clean columns
    nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]
    
    print('{} venues were returned by Foursquare for {}.'.format(nearby_venues.shape[0], Stations_Location.iloc[i,1]))
    
    nearby_venues_parent = nearby_venues.merge(CategoryKey, on = 'categories')
    
    Station_dfs[Stations_Location.iloc[i,1]] =nearby_venues_parent.groupby(['Parent']).size()
    
    

105 venues were returned by Foursquare for Blair.
104 venues were returned by Foursquare for Cyrville.
195 venues were returned by Foursquare for St-Laurent.
85 venues were returned by Foursquare for Tremblay.
125 venues were returned by Foursquare for Hurdman.
107 venues were returned by Foursquare for Lees.
189 venues were returned by Foursquare for uOttawa.
194 venues were returned by Foursquare for Rideau.
191 venues were returned by Foursquare for Lyon.
188 venues were returned by Foursquare for Pimisi.
184 venues were returned by Foursquare for Bayview.
200 venues were returned by Foursquare for Bloor-Yonge.
190 venues were returned by Foursquare for Wellesley.
175 venues were returned by Foursquare for College.
199 venues were returned by Foursquare for Dundas.
198 venues were returned by Foursquare for Queen.
198 venues were returned by Foursquare for King.
97 venues were returned by Foursquare for Union.
200 venues were returned by Foursquare for St-Andrew.
190 venues were ret

For Each Station Translate Venue Sub-Categories to Parent Category Based on Key

In [6]:
keys = list(Station_dfs.keys())

Stations_VenueCounts = pd.Series([])

for i in range(len(keys)):
    #print(i)
    #print(Station_dfs[keys[i]])
    Stations_VenueCounts = pd.concat([Stations_VenueCounts, Station_dfs[keys[i]]], axis=1, sort=True)
    #print(Stations_VenueCounts)

Stations_VenueCounts

,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
Arts & Entertainment,NaN,1.0,2.0,3.0,2.0,2.0,3.0,2.0,4.0,2.0,24,10,2.0,6.0,6.0,15.0,9.0,4.0,4.0,4.0,11.0,3.0,3.0,37.0,11.0,4.0,3.0
College & University,NaN,1.0,NaN,1.0,1.0,11.0,13.0,112.0,2.0,4.0,2,1,5.0,2.0,3.0,6.0,NaN,2.0,NaN,3.0,1.0,5.0,19.0,36.0,23.0,12.0,NaN
Event,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,1,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN
Food,NaN,26.0,8.0,35.0,16.0,15.0,15.0,14.0,28.0,25.0,34,25,25.0,41.0,36.0,59.0,36.0,30.0,23.0,39.0,32.0,46.0,35.0,14.0,12.0,25.0,28.0
Nightlife Spot,NaN,1.0,2.0,NaN,NaN,3.0,3.0,6.0,10.0,4.0,2,11,2.0,10.0,4.0,5.0,6.0,7.0,4.0,6.0,8.0,2.0,1.0,4.0,4.0,5.0,5.0
Outdoors & Recreation,NaN,4.0,2.0,1.0,4.0,14.0,11.0,9.0,50.0,7.0,21,10,5.0,5.0,6.0,10.0,3.0,7.0,1.0,4.0,16.0,4.0,3.0,11.0,4.0,6.0,6.0
Professional & Other Places,NaN,20.0,30.0,7.0,13.0,16.0,15.0,18.0,21.0,82.0,30,42,92.0,26.0,42.0,41.0,31.0,84.0,17.0,97.0,61.0,82.0,80.0,34.0,37.0,43.0,68.0
Residence,NaN,NaN,2.0,NaN,NaN,14.0,9.0,3.0,2.0,4.0,12,9,1.0,17.0,11.0,NaN,1.0,1.0,NaN,1.0,7.0,11.0,NaN,NaN,3.0,4.0,1.0
Shop & Service,NaN,35.0,36.0,133.0,20.0,15.0,10.0,10.0,53.0,21.0,28,51,49.0,61.0,44.0,47.0,98.0,37.0,12.0,28.0,15.0,28.0,23.0,14.0,13.0,24.0,34.0
Travel & Transport,NaN,10.0,11.0,1.0,24.0,26.0,16.0,5.0,8.0,28.0,16,11,8.0,11.0,10.0,5.0,7.0,14.0,29.0,12.0,25.0,8.0,6.0,5.0,7.0,61.0,5.0


### Fix Errors and Clean Dataframe

In [7]:
## Column Names ##
# Give a title to first column
col01 = 'col01'

# Create a list of column name for the Stations_VenueCounts dataframe
column_names = [col01] + keys

# Assign column names to Stations_VenueCounts dataframe
Stations_VenueCounts.columns = [column_names]


## Drop First Empty Column ##
Stations_VenueCounts = Stations_VenueCounts.drop('col01', axis=1)

    
## Transpose the dataframe ##
Stations_VenueCounts = Stations_VenueCounts.transpose()


## Drop Events Column. Greater than 25% NaNs ##
Stations_VenueCounts = Stations_VenueCounts.drop('Event', axis=1)


## Drop Residence Column. Greater than 25% NaNs ##
Stations_VenueCounts = Stations_VenueCounts.drop('Residence', axis=1)


## Replace NaN with 0s
Stations_VenueCounts = Stations_VenueCounts.fillna(0)

C:\DataScience\lib\site-packages\pandas\core\generic.py:3111: PerformanceWarning: dropping on a non-lexsorted multi-index without a level parameter may impact performance.
  obj = obj._drop_axis(labels, axis, level=level, errors=errors)


In [8]:
Stations_VenueCounts

,Arts & Entertainment,College & University,Food,Nightlife Spot,Outdoors & Recreation,Professional & Other Places,Shop & Service,Travel & Transport
Blair,1.0,1.0,26.0,1.0,4.0,20.0,35.0,10.0
Cyrville,2.0,0.0,8.0,2.0,2.0,30.0,36.0,11.0
St-Laurent,3.0,1.0,35.0,0.0,1.0,7.0,133.0,1.0
Tremblay,2.0,1.0,16.0,0.0,4.0,13.0,20.0,24.0
Hurdman,2.0,11.0,15.0,3.0,14.0,16.0,15.0,26.0
Lees,3.0,13.0,15.0,3.0,11.0,15.0,10.0,16.0
uOttawa,2.0,112.0,14.0,6.0,9.0,18.0,10.0,5.0
Rideau,4.0,2.0,28.0,10.0,50.0,21.0,53.0,8.0
Lyon,2.0,4.0,25.0,4.0,7.0,82.0,21.0,28.0
Pimisi,24.0,2.0,34.0,2.0,21.0,30.0,28.0,16.0


### Export Final Dataframe

In [9]:
Stations_VenueCounts.to_csv('Stations_VenueCounts.csv')